In [ ]:
import ROOT
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.gridspec as gridspec
import math
import uproot
from array import array

In [ ]:

# Most Stats Available
data_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wiremod_ndwidths_data_xf_6k.root"
mc_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wiremod_ndwidths_mc_xf_6k.root"

print("Got MC and Data Files to compare and analyze")

In [ ]:
rfile_data = ROOT.TFile.Open(data_file)
rfile_mc = ROOT.TFile.Open(mc_file)

DATA_TAG = "v10_06_00_02"

rfile_data.ls()

# Iterative Truncated Mean Function

In [ ]:
def iterative_truncated_mean(h, sig_down, sig_up, tol):
    #initialize the final result
    result_mean = 0
    result_std = 0

    mean = h.GetMean()
    std_dev = h.GetRMS()
    if ROOT.TMath.Abs(result_mean - mean) < tol:
        return [result_mean, result_std]
    
    result_mean = mean
    result_std = std_dev

    flag=True
    h_prev = h.Clone("h_prev")
    i=1
    while flag:
        quantile_fractions = array('d', [0.5])
        quantile_values = array('d', [0.0]*len(quantile_fractions))
        h_prev.GetQuantiles(len(quantile_fractions), quantile_values, quantile_fractions)

        median = quantile_values[0]
        h_new = h_prev.Clone("h_new")
        h_new.Reset()
        for bin in range(1, h_prev.GetNbinsX() + 1):
            if h_prev.GetBinLowEdge(bin) > median + sig_up * std_dev or h_prev.GetBinLowEdge(bin) +h_prev.GetBinWidth(bin) < median - sig_down * std_dev:
                continue
            h_new.SetBinContent(bin, h_prev.GetBinContent(bin))
            h_new.SetBinError(bin, h_prev.GetBinError(bin))
        
        mean = h_new.GetMean()
        std_dev = h_new.GetRMS()
        if ROOT.TMath.Abs(result_mean - mean) < tol:
            flag = False
            return [result_mean, result_std]
        
        result_mean = mean
        result_std = std_dev
        h_prev.SetDirectory(0)
        h_prev.Delete()
        h_prev = h_new.Clone("h_prev")
        i += 1
        if i > 100:
            print("Warning: Iteration limit reached, returning last computed mean and std_dev")
            return [result_mean, result_std]
        
print("Iterative truncated mean function defined")

In [ ]:
# Function to profile a 2D histogram along X axis and compute iterative truncated mean for each slice

def profile_2d_histogram(file_name, dim, tpc, plane):
    means = []
    errors = []
    idx = 3*tpc+plane
    rfile = ROOT.TFile.Open(file_name)
    h = rfile.Get("hwidth"+str(idx))

    # Loop over the bins in the selected dimension
    for bin in range(1, h.GetAxis(dim).GetNbins() + 1):
        h.GetAxis(dim).SetRange(bin, bin)
        h_1d_temp = h.Projection(6)
        result = iterative_truncated_mean(h_1d_temp, 2, 1.75, 1e-4)
        means.append(result[0])
        h_1d_temp.Delete()
        #errors.append(result[1]/math.sqrt(h_1d_temp.GetEntries()))
        errors.append(result[1])
    
    return means, errors

print("Profiling function defined")                  

In [ ]:
# I want to convert a ROOT TH2D to a plt.hist2d for better plotting
def root_th2d_to_plt_hist2d(th2d):
    # Get the bin edges and contents from the ROOT TH2D
    x_edges = [th2d.GetXaxis().GetBinLowEdge(i) for i in range(1, th2d.GetNbinsX() + 2)]
    y_edges = [th2d.GetYaxis().GetBinLowEdge(i) for i in range(1, th2d.GetNbinsY() + 2)]
    z_values = np.zeros((th2d.GetNbinsX(), th2d.GetNbinsY()))

    for i in range(1, th2d.GetNbinsX() + 1):
        for j in range(1, th2d.GetNbinsY() + 1):
            z_values[i-1, j-1] = th2d.GetBinContent(i, j)

    return np.array(x_edges), np.array(y_edges), np.array(z_values)

# Example usage:
# th2d = rfile_data.Get("hwidth0")
# x_edges, y_edges, z_values = root_th2d_to_plt_hist2d(th2d)
# plt.hist2d(x_edges[:-1], y_edges[:-1], z_values.T, bins=[x_edges, y_edges], cmap='viridis')
# plt.colorbar(label='Counts')
# plt.xlabel('X-axis label')                
#plt.ylabel('Y-axis label')
# plt.title('2D Histogram from ROOT TH2D')
# plt.show()    

# I want to convert a ROOT TH1D with errors to a plt.errorbar for better plotting
def root_th1d_to_plt_errorbar(th1d):
    # Get the bin centers, contents, and errors from the ROOT TH1D
    x_values = [th1d.GetBinCenter(i) for i in range(1, th1d.GetNbinsX() + 1)]
    y_values = [th1d.GetBinContent(i) for i in range(1, th1d.GetNbinsX() + 1)]
    y_errors = [th1d.GetBinError(i) for i in range(1, th1d.GetNbinsX() + 1)]

    return np.array(x_values), np.array(y_values), np.array(y_errors) 

# Example usage:
# th1d = rfile_data.Get("hwidth0").ProjectionX()
# x_values, y_values, y_errors = root_th1d_to_plt_errorbar(th1d)
# plt.errorbar(x_values, y_values, yerr=y_errors, fmt='o', ecolor='red', capsize=5)
# plt.xlabel('X-axis label')                
# plt.ylabel('Y-axis label')
# plt.title('1D Histogram with Error Bars from ROOT TH1D')
# plt.show()


# Function to create a ROOT TH1D from means and errors
def make_mean_hist(h2d, means, errors):
    h_means = h2d.ProjectionX()
    h_means.Reset()
    for i in range(1, h_means.GetNbinsX() + 1):
        h_means.SetBinContent(i, means[i-1])
        h_means.SetBinError(i, errors[i-1])
    
    return h_means

In [ ]:
dim_dict = {0: "x", 1: "y", 2: "z", 3:"txz", 4:"tyz", 5:"dqdx"}
label_dict = {0: "Reconstructed Hit X [cm]", 
              1: "Reconstructed Hit Y [cm]", 
              2: "Reconstructed Hit Z [cm]", 
              3:"Reconstructed Hit "+r"$\theta_{XZ}$ [degrees]", 
              4:"Reconstructed Hit "+r"$\theta_{YZ}$ [degrees]", 
              5:"Reconstructed Hit dQ/dx [ADC/cm]"}

In [ ]:

def plot_profiles_with_itms(DIM, isData=True, xlim0=None, xlim1=None, DATA_TAG=None):

    kNtpcs = 2
    kNplanes = 3

    fig, axs = plt.subplots(kNtpcs, kNplanes, figsize=(12, 8), sharey=True)      
    

    for tpc in range(kNtpcs):
        for plane in range(kNplanes):
            idx = 3*tpc + plane
            #thn = rfile_data.Get("hwidth"+str(idx))
            h = 0
            if isData:
                h = rfile_data.Get("hwidth"+str(idx))
            else:
                h = rfile_mc.Get("hwidth"+str(idx))

            #means, errors = profile_2d_histogram(data_file, DIM, tpc, plane)
            ax = axs[tpc, plane]
            # Loop over the bins in the selected dimension
            for bin in range(1, h.GetAxis(DIM).GetNbins() + 1):
                if bin % 2 != 0:  # Skip odd bins
                    continue
                binx_val = h.GetAxis(DIM).GetBinCenter(bin) 
                if tpc == 0 and (binx_val < -200 or binx_val > 0):
                    continue
                if tpc == 1 and (binx_val < 0 or binx_val > 200):
                    continue
                
                h.GetAxis(DIM).SetRange(bin, bin)
                h_1d_temp = h.Projection(6)
                if h_1d_temp.GetEntries() < 100:
                    h_1d_temp.Delete()
                    continue
                h_1d_temp.Scale(1./h_1d_temp.Integral())
                x, y, yerr = root_th1d_to_plt_errorbar(h_1d_temp)
                ax.errorbar(x, y, xerr=np.ones_like(yerr)*((x[1]-x[0])/2), 
                        yerr=yerr, fmt='o', capsize=0, label='bin: '+str(bin), markersize=0)
                
                h_1d_temp.Delete()

            if tpc == 0:
                if xlim0 is not None:
                    ax.set_xlim(xlim0)
            else:
                if xlim1 is not None:
                    ax.set_xlim(xlim1)

            
                
            #im = ax.hist2d(x_edges, y_edges, z_values.T, cmap='viridis')
            ax.set_title(f'TPC {tpc}, Plane {plane}')
            #fig.colorbar(im[3], ax=ax, label=''

            h.Delete()
            ax.legend()
         
    
    fig.supxlabel("Reconstructed Hit Width [ticks]", fontsize=20)
    fig.supylabel("Hit Counts: Normalized", fontsize=20)
  
    t = "SBND Off-Beam "
    if isData:
        t += "Data: "
    else:
        t += "MC: "
    if DATA_TAG is not None:
        t += DATA_TAG
    t += " - " + label_dict[DIM]

    fig.suptitle(t, fontsize=20)
    #cbar_ax = fig.add_subplot(gs[0, 2])
    #fig.colorbar(pcm, cax=cbar_ax)
    #fig.colorbar(pcm, ax=axs, fraction=.05, label='')
    plt.tight_layout()
    plt.show()

print("Defining plot function")

In [ ]:
plot_profiles_with_itms(0, isData=True, xlim0=(0, 10), xlim1=(0, 10), DATA_TAG=DATA_TAG)